# Why Python? A Hands-On Comparison with JASP

You just learned how to run statistical tests in JASP. Now let's do the **exact same tests** in Python and see what we gain.

## Why bother with code when JASP exists?

JASP is excellent for learning statistical concepts and running quick analyses. Python offers additional capabilities that become valuable as your research grows:

| | JASP (GUI) | Python (Code) |
|---|---|---|
| **Reproducibility** | Need to remember which buttons you clicked | Save code, run again anytime — same result guaranteed |
| **Automation** | Analyze 50 brain regions? Click 50 times | `for region in regions: analyze(region)` — done in seconds |
| **Flexibility** | Limited to built-in options | Create any analysis you can describe |
| **Transparency** | Results appear; steps are hidden | You control (and can inspect) each step |
| **Sharing** | Colleague needs JASP installed | Share a .py file or notebook — runs in any Python environment |
| **Scale** | Great for one dataset at a time | Process thousands of files overnight |

**Bottom line:** JASP is great for learning concepts and quick checks. Python gives you power and flexibility for real research.

Let's prove it. We'll replicate everything you just did in JASP — and then go further.

---

## Part 1: Setup and Loading Data

First, let's load the Python libraries we need.

In [ ]:
# Step 1: Import the libraries we'll use
import pandas as pd          # For working with data tables
import numpy as np           # For numerical operations
import matplotlib.pyplot as plt  # For creating plots
import scipy.stats as stats  # For statistical tests

print("Libraries loaded successfully!")

### The Study

A pharmacology experiment examining how dopaminergic drugs affect locomotor activity in rodents.

**Dependent Variable**: Number of squares entered in an open-field test (measure of locomotion)

**Independent Variables** (8 drug treatment groups):
- **Control**: No drug (baseline)
- **Amph**: Amphetamine (dopamine releaser → increases locomotion)
- **Res only**: Reserpine (depletes dopamine → decreases locomotion)
- **Res+Amph**: Can amphetamine overcome reserpine's effects?
- **Res+MT**: Reserpine + alpha-methyltyrosine (blocks dopamine synthesis)
- **Res+MT+Amph**: Triple combination
- **Res+MT+DOPA**: L-DOPA (dopamine precursor) to restore function?
- **Res+MT+Amph+DOPA**: Maximum restoration attempt

### Loading Data: The Peek-Then-Use Pattern

**Good habit:** Always look at your data before analyzing it! This helps you catch problems early.

In [ ]:
# Step 2: Load the ANOVA dataset
anova_df = pd.read_excel('class_data_undergrad.xlsx')

# Step 3: ALWAYS peek at your data first!
anova_df.head()

In [ ]:
# Step 4: Check the structure of your data
print("DATA QUALITY CHECK")
print("=" * 40)
print(f"Number of rows: {len(anova_df)}")
print(f"Number of columns: {len(anova_df.columns)}")
print(f"\nColumn names: {list(anova_df.columns)}")
print(f"\nMissing values: {anova_df.isna().sum().sum()}")
print(f"\nGroups in the data: {anova_df['Group'].unique()}")

### Visualize Before Analyzing!

**Good habit:** Always plot your data before running statistics. Visualizations help you:
- Spot outliers or data entry errors
- See the pattern before confirming it with numbers
- Choose the right statistical test

In [ ]:
# Step 5: Visualize the data — immediate payoff!
plt.figure(figsize=(12, 5))
anova_df.boxplot(column='Squares entered', by='Group', rot=45)
plt.title('Locomotor Activity by Treatment Group')
plt.suptitle('')  # Remove the automatic title pandas adds
plt.ylabel('Squares Entered')
plt.xlabel('Treatment Group')
plt.tight_layout()
plt.show()

print("What patterns do you notice? Which groups look different from Control?")

In [ ]:
# Step 6: Descriptive statistics by group
# This is what JASP calls "Descriptives"
descriptives = anova_df.groupby('Group')['Squares entered'].agg(['count', 'mean', 'std']).round(2)
descriptives.columns = ['N', 'Mean', 'SD']

# Order groups logically for pharmacological interpretation
group_order = ['Control', 'Amph', 'Res only', 'Res+Amph', 'Res+MT', 'Res+MT+Amph', 'Res+MT+DOPA', 'Res+MT+Amph+DOPA']
descriptives = descriptives.reindex(group_order)

print("Descriptive Statistics by Treatment Group")
print("=" * 50)
print(descriptives.to_string())
print("\nCompare with your JASP output!")

---

### 🎯 Your Turn: Load the T-Test Data

Now it's your turn! Load the file `Old and New Data Set t-tests.xlsx` (sheet: `Data Set 1`) and look at the first few rows with `.head()`.

**Hint:** You'll need to use `pd.read_excel()` with a `sheet_name` argument.

<details>
<summary>Click to reveal answer</summary>

```python
ttest_df1 = pd.read_excel('Old and New Data Set t-tests.xlsx',
                          sheet_name='Data Set 1',
                          skiprows=1)  # Skip the empty first row
ttest_df1.head()
```

</details>

In [ ]:
# Your code here:


---

## Part 2: T-Tests

We'll analyze both datasets from your JASP session and compare independent vs. paired t-tests.

### Data Set 1: Loading and Cleaning

This Excel file has a quirky structure — let's see what we're dealing with.

In [ ]:
# Load the t-test data and peek at it
df1_raw = pd.read_excel('Old and New Data Set t-tests.xlsx', sheet_name='Data Set 1', header=None)

print("Raw data (first 18 rows):")
print(df1_raw.head(18))
print("\n... and the last few rows:")
print(df1_raw.tail(5))

**What we see:** The actual data starts at row 2, and there are summary statistics at the bottom that we need to exclude.

Let's extract just the data we need:

In [ ]:
# Extract the New and Old columns (skip header row, get numeric values only)
new_data1 = pd.to_numeric(df1_raw.iloc[2:, 0], errors='coerce').dropna().values
old_data1 = pd.to_numeric(df1_raw.iloc[2:, 1], errors='coerce').dropna().values

print("T-Test Data Set 1 (Equal Variances):")
print(f"  New group: n={len(new_data1)}, mean={new_data1.mean():.2f}, SD={new_data1.std(ddof=1):.2f}")
print(f"  Old group: n={len(old_data1)}, mean={old_data1.mean():.2f}, SD={old_data1.std(ddof=1):.2f}")

### Visualize Before Testing

Before running the t-test, let's see what the data looks like:

In [ ]:
# Visualize the two groups
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Histogram
axes[0].hist(new_data1, bins=8, alpha=0.7, label='New', color='steelblue', edgecolor='black')
axes[0].hist(old_data1, bins=8, alpha=0.7, label='Old', color='coral', edgecolor='black')
axes[0].set_xlabel('Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Data Set 1: Overlapping Histograms')
axes[0].legend()

# Boxplot
axes[1].boxplot([new_data1, old_data1], labels=['New', 'Old'])
axes[1].set_ylabel('Score')
axes[1].set_title('Data Set 1: Side-by-Side Boxplots')

plt.tight_layout()
plt.show()

print("The groups look fairly similar in spread (variance). Is the New group higher?")

### Independent Samples T-Test

**Question**: Is there a difference between the New and Old groups?

In JASP: T-Tests → Independent Samples T-Test → drag variables → click options...

In Python: One line of code!

In [ ]:
# Independent samples t-test (assumes equal variances)
t_stat, p_value = stats.ttest_ind(new_data1, old_data1)

print("INDEPENDENT SAMPLES T-TEST (Data Set 1)")
print("=" * 45)
print(f"New group mean: {new_data1.mean():.2f} (n={len(new_data1)})")
print(f"Old group mean: {old_data1.mean():.2f} (n={len(old_data1)})")
print(f"\nt = {t_stat:.3f}, p = {p_value:.4f}")
print(f"\nSignificant at α=0.05? {'Yes' if p_value < 0.05 else 'No'}")

### Teaching Moment: Independent vs. Paired T-Tests

What happens if we run the same data as a **paired** t-test? The results will differ!

- **Independent t-test**: Assumes the groups are unrelated (different subjects)
- **Paired t-test**: Assumes each observation in one group is matched to one in the other

---

⚠️ **Important Warning:** The following is a "what happens if we use the wrong test" demonstration. A paired t-test requires **true subject-level pairing** (e.g., the same person measured before and after treatment). We're artificially pairing unrelated observations here to show how results change — **never do this with real data!**

In [ ]:
# For paired t-test, we need equal sample sizes
# Truncate to the smaller group (this is artificial pairing - just for demonstration!)
n_min = min(len(new_data1), len(old_data1))
new_paired = new_data1[:n_min]
old_paired = old_data1[:n_min]

# Independent t-test
t_ind, p_ind = stats.ttest_ind(new_paired, old_paired)

# Paired t-test (ttest_rel = related samples)
t_paired, p_paired = stats.ttest_rel(new_paired, old_paired)

print("COMPARISON: Independent vs. Paired T-Tests")
print("=" * 50)
print(f"Using first {n_min} observations from each group\n")
print(f"Independent t-test: t = {t_ind:.3f}, p = {p_ind:.4f}")
print(f"Paired t-test:      t = {t_paired:.3f}, p = {p_paired:.4f}")
print("\n** Key insight: The p-values differ! **")
print("\nWhen to use each:")
print("  - Independent: Different subjects in each group (e.g., treatment vs control)")
print("  - Paired: Same subjects measured twice (e.g., before vs after)")

---

### 🎯 Your Turn: Create a Histogram

Create a histogram showing just the `new_data1` values. Use `plt.hist()` with:
- `bins=10`
- `color='steelblue'`
- `edgecolor='black'`

Don't forget to add a title with `plt.title()` and show the plot with `plt.show()`!

<details>
<summary>Click to reveal answer</summary>

```python
plt.hist(new_data1, bins=10, color='steelblue', edgecolor='black')
plt.title('Distribution of New Group Scores')
plt.xlabel('Score')
plt.ylabel('Count')
plt.show()
```

</details>

In [ ]:
# Your code here:


---

### Data Set 2: Unequal Variances — When Standard T-Tests Fail

**Problem**: The standard t-test assumes equal variances. But sometimes groups have very different spreads. Let's load Data Set 2 and see:

In [ ]:
# Load Data Set 2 (note: sheet name has trailing space in the Excel file)
df2_raw = pd.read_excel('Old and New Data Set t-tests.xlsx', sheet_name='Data Set 2 ', header=None)
new_data2 = pd.to_numeric(df2_raw.iloc[2:, 0], errors='coerce').dropna().values
old_data2 = pd.to_numeric(df2_raw.iloc[2:, 1], errors='coerce').dropna().values

print("T-Test Data Set 2 (Unequal Variances):")
print(f"  New group: n={len(new_data2)}, mean={new_data2.mean():.2f}, SD={new_data2.std(ddof=1):.2f}")
print(f"  Old group: n={len(old_data2)}, mean={old_data2.mean():.2f}, SD={old_data2.std(ddof=1):.2f}")

In [ ]:
# VISUALIZE the variance difference — much clearer than numbers!
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].hist(new_data2, bins=8, alpha=0.7, label='New', color='steelblue', edgecolor='black')
axes[0].hist(old_data2, bins=8, alpha=0.7, label='Old', color='coral', edgecolor='black')
axes[0].set_xlabel('Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Data Set 2: Overlapping Histograms')
axes[0].legend()

axes[1].boxplot([new_data2, old_data2], labels=['New', 'Old'])
axes[1].set_ylabel('Score')
axes[1].set_title('Data Set 2: Side-by-Side Boxplots')

plt.tight_layout()
plt.show()

print("Notice how the 'New' group is MUCH more spread out than the 'Old' group!")

In [ ]:
# Check variances numerically
var_new = new_data2.var(ddof=1)
var_old = old_data2.var(ddof=1)

print("Variance Check (Data Set 2):")
print(f"  New group: variance = {var_new:.2f}, SD = {np.sqrt(var_new):.2f}")
print(f"  Old group: variance = {var_old:.2f}, SD = {np.sqrt(var_old):.2f}")
print(f"\nVariance ratio: {var_new/var_old:.2f}x difference!")

# Levene's test for equality of variances
lev_stat, lev_p = stats.levene(new_data2, old_data2)
print(f"\nLevene's test: F = {lev_stat:.3f}, p = {lev_p:.4f}")
if lev_p > 0.05:
    print("Equal variances? Yes (p > 0.05)")
else:
    print("Equal variances? No (p < 0.05) — use Welch's t-test!")

In [ ]:
# Compare standard vs Welch's t-test
t_standard, p_standard = stats.ttest_ind(new_data2, old_data2, equal_var=True)
t_welch, p_welch = stats.ttest_ind(new_data2, old_data2, equal_var=False)  # Welch's

print("T-TEST COMPARISON (Data Set 2 - Unequal Variances)")
print("=" * 55)
print(f"New group: mean = {new_data2.mean():.2f}, SD = {new_data2.std(ddof=1):.2f}")
print(f"Old group: mean = {old_data2.mean():.2f}, SD = {old_data2.std(ddof=1):.2f}")
print(f"\nStandard t-test (assumes equal var): t = {t_standard:.3f}, p = {p_standard:.4f}")
print(f"Welch's t-test  (unequal var OK):    t = {t_welch:.3f}, p = {p_welch:.4f}")
print("\n** When variances are unequal, use Welch's t-test! **")
print("In Python: stats.ttest_ind(a, b, equal_var=False)")

---

## Part 3: One-Way ANOVA

**Question**: Does locomotor activity differ across the 8 drug treatment groups?

In JASP: ANOVA → drag Dependent Variable → drag Fixed Factors → check Post Hoc...

In Python: First visualize, then test!

In [ ]:
# Always visualize before running stats!
plt.figure(figsize=(12, 5))
anova_df.boxplot(column='Squares entered', by='Group', rot=45)
plt.title('What patterns do you see?')
plt.suptitle('')  # Remove the automatic pandas title
plt.ylabel('Squares Entered')
plt.xlabel('Treatment Group')
plt.tight_layout()
plt.show()

print("Look at the plot: Which groups appear different from Control?")
print("Which groups have near-zero locomotion?")

In [ ]:
# Separate the groups for analysis
groups = {}
for group_name in group_order:
    groups[group_name] = anova_df[anova_df['Group'] == group_name]['Squares entered'].values

# One-way ANOVA
F_stat, p_value = stats.f_oneway(*groups.values())

# Calculate degrees of freedom
k = len(groups)  # number of groups
N = len(anova_df)  # total sample size
df_between = k - 1
df_within = N - k

# Format p-value properly (don't hard-code!)
if p_value < 0.001:
    p_str = "p < .001"
else:
    p_str = f"p = {p_value:.4f}"

print("ONE-WAY ANOVA: Locomotor Activity by Treatment Group")
print("=" * 55)
print(f"\nF({df_between},{df_within}) = {F_stat:.2f}, {p_str}")
print("\n** Compare with JASP: F(7,102) = 82.00, p < .001 **")
print("\nGroup means (squares entered):")
for group_name in group_order:
    mean = groups[group_name].mean()
    print(f"  {group_name:20s}: {mean:6.2f}")

---

## Part 4: Post-Hoc Comparisons (Tukey HSD)

The ANOVA tells us groups differ, but **which** groups differ from each other?

With 8 groups, we have 8×7/2 = **28 pairwise comparisons**.

### Why Not Just Run 28 T-Tests?

You might think: "If I want to compare 8 groups, I'll just run all pairwise t-tests!"

**The problem:** With 28 comparisons at α = 0.05, you expect ~1-2 false positives by chance alone. This is called the **multiple comparisons problem**.

**Solutions:**
1. **Tukey HSD** (what we'll use) — adjusts p-values to control family-wise error rate
2. **Bonferroni correction** — multiply each p-value by the number of tests (more conservative)

Let's use Tukey HSD:

In [ ]:
from scipy.stats import tukey_hsd

# Tukey HSD for all pairwise comparisons
result = tukey_hsd(*[groups[g] for g in group_order])

print("TUKEY HSD POST-HOC COMPARISONS")
print("=" * 70)
print(f"{'Comparison':<45} {'Mean Diff':>10} {'p-value':>10}")
print("-" * 70)

# Store significant and non-significant comparisons
significant_pairs = []
nonsig_pairs = []

for i in range(len(group_order)):
    for j in range(i+1, len(group_order)):
        g1, g2 = group_order[i], group_order[j]
        mean_diff = groups[g1].mean() - groups[g2].mean()
        p_val = result.pvalue[i, j]
        
        sig_marker = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
        comparison = f"{g1} vs {g2}"
        print(f"{comparison:<45} {mean_diff:>10.2f} {p_val:>10.3f} {sig_marker}")
        
        if p_val < 0.05:
            significant_pairs.append((g1, g2, mean_diff, p_val))
        else:
            nonsig_pairs.append((g1, g2, mean_diff, p_val))

print("-" * 70)
print(f"Significant comparisons: {len(significant_pairs)} of 28")
print("*** p < .001, ** p < .01, * p < .05")

In [ ]:
# Highlight key pharmacological findings — using COMPUTED p-values from Tukey results
print("\nKEY PHARMACOLOGICAL FINDINGS")
print("=" * 60)

# Helper function to format p-values
def format_p(p):
    if p < 0.001:
        return "p < .001"
    else:
        return f"p = {p:.3f}"

# Get p-values from Tukey result matrix
# Indices match group_order: Control=0, Amph=1, Res only=2, etc.
p_amph_ctrl = result.pvalue[0, 1]  # Control vs Amph
p_res_ctrl = result.pvalue[0, 2]   # Control vs Res only
p_resamph_ctrl = result.pvalue[0, 3]  # Control vs Res+Amph
p_final_ctrl = result.pvalue[0, 7]  # Control vs Res+MT+Amph+DOPA

print("\n1. Amphetamine INCREASES locomotion:")
amph_vs_ctrl = groups['Amph'].mean() - groups['Control'].mean()
print(f"   Amph vs Control: +{amph_vs_ctrl:.1f} squares ({format_p(p_amph_ctrl)})")

print("\n2. Reserpine ABOLISHES locomotion:")
res_vs_ctrl = groups['Res only'].mean() - groups['Control'].mean()
print(f"   Res only vs Control: {res_vs_ctrl:.1f} squares ({format_p(p_res_ctrl)})")

print("\n3. Amphetamine PARTIALLY restores function after reserpine:")
print(f"   Res+Amph mean: {groups['Res+Amph'].mean():.1f} (vs Control: {format_p(p_resamph_ctrl)})")

print("\n4. MT blocks the amphetamine restoration:")
print(f"   Res+MT+Amph mean: {groups['Res+MT+Amph'].mean():.1f} (essentially zero)")

print("\n5. L-DOPA + Amph can restore function even with MT:")
print(f"   Res+MT+Amph+DOPA mean: {groups['Res+MT+Amph+DOPA'].mean():.1f} (vs Control: {format_p(p_final_ctrl)})")

print("\n** These findings demonstrate the dopamine hypothesis of locomotion! **")

---

### 🎯 Your Turn: Run a T-Test

Run an independent samples t-test comparing the `Control` group to the `Res only` group.

Use `stats.ttest_ind()` with the data from the `groups` dictionary we created earlier.

**Hint:** Access the data with `groups['Control']` and `groups['Res only']`

<details>
<summary>Click to reveal answer</summary>

```python
t_stat, p_val = stats.ttest_ind(groups['Control'], groups['Res only'])
print(f"t = {t_stat:.3f}, p = {p_val:.4f}")
print(f"Control mean: {groups['Control'].mean():.2f}")
print(f"Res only mean: {groups['Res only'].mean():.2f}")
```

</details>

In [ ]:
# Your code here:


---

## Part 5: Things Python Can Do That JASP Can't

### Advantage 1: Automation

What if you needed to run pairwise comparisons for multiple outcome measures? In JASP, you'd click through menus dozens of times. In Python, it's a loop:

In [ ]:
# Run all 28 pairwise t-tests programmatically
print("AUTOMATED PAIRWISE T-TESTS (all 28 comparisons)")
print("=" * 55)

results_list = []
for i in range(len(group_order)):
    for j in range(i+1, len(group_order)):
        g1, g2 = group_order[i], group_order[j]
        t, p = stats.ttest_ind(groups[g1], groups[g2])
        results_list.append({
            'Group 1': g1,
            'Group 2': g2,
            't': t,
            'p': p,
            'significant': p < 0.05
        })

results_df = pd.DataFrame(results_list)
print(f"\nRan {len(results_df)} t-tests in a loop.")
print(f"Significant results (uncorrected): {results_df['significant'].sum()}")

# Apply Bonferroni correction
results_df['p_bonf'] = results_df['p'] * len(results_df)
results_df['p_bonf'] = results_df['p_bonf'].clip(upper=1.0)  # Cap at 1.0
results_df['sig_bonf'] = results_df['p_bonf'] < 0.05
print(f"Significant after Bonferroni correction: {results_df['sig_bonf'].sum()}")

print("\n⚠️ Note: Running many uncorrected t-tests inflates false positives!")
print("   Always use Tukey HSD or Bonferroni correction for post-hoc comparisons.")
print("\n--> In JASP, you'd click through 28 separate comparisons.")
print("    In Python, 8 lines of code.")

### Advantage 2: Custom Visualizations

JASP gives you canned plots. Python gives you full control.

In [ ]:
# Publication-quality bar plot with error bars
fig, ax = plt.subplots(figsize=(12, 6))

means = [groups[g].mean() for g in group_order]
sems = [groups[g].std() / np.sqrt(len(groups[g])) for g in group_order]

# Color code by drug effect
colors = ['#4DAF4A',   # Control - green
          '#E41A1C',   # Amph - red (stimulant)
          '#377EB8',   # Res only - blue (depleted)
          '#FF7F00',   # Res+Amph - orange (partial restoration)
          '#377EB8',   # Res+MT - blue
          '#377EB8',   # Res+MT+Amph - blue
          '#377EB8',   # Res+MT+DOPA - blue
          '#FF7F00']   # Res+MT+Amph+DOPA - orange (restoration)

bars = ax.bar(range(len(group_order)), means, yerr=sems, 
              color=colors, edgecolor='black', capsize=4, alpha=0.8)

ax.set_xticks(range(len(group_order)))
ax.set_xticklabels(group_order, rotation=45, ha='right', fontsize=10)
ax.set_ylabel('Squares Entered (± SEM)', fontsize=12)
ax.set_xlabel('Treatment Group', fontsize=12)

# Use computed F and p for title
if p_value < 0.001:
    title_p = "p < .001"
else:
    title_p = f"p = {p_value:.3f}"
ax.set_title(f'Locomotor Activity by Pharmacological Treatment\nF({df_between},{df_within}) = {F_stat:.2f}, {title_p}', fontsize=14)

# Add horizontal line at Control mean for reference
ax.axhline(y=groups['Control'].mean(), color='gray', linestyle='--', alpha=0.5, label='Control baseline')
ax.legend()

plt.tight_layout()
plt.show()

print("\nColor key: Green = Control, Red = Stimulant effect, Blue = Depleted, Orange = Restored")

### Advantage 3: Bootstrap Confidence Intervals

This isn't even an option in JASP's menus. In Python, it's straightforward.

### Why Bootstrap?

Traditional statistics assume your data follows a specific distribution (usually normal). But what if it doesn't? Or what if you have a small sample?

**Bootstrapping** lets you estimate uncertainty without those assumptions:
1. Resample your data (with replacement) thousands of times
2. Calculate your statistic each time
3. The spread of results IS your confidence interval

This is especially useful when:
- Sample sizes are small
- Data is skewed or has outliers
- You want to be confident your results aren't flukes

In [ ]:
# Bootstrap CI for Amph vs Control difference
amph = groups['Amph']
control = groups['Control']

n_bootstrap = 10000
boot_diffs = []

np.random.seed(42)  # For reproducibility
for _ in range(n_bootstrap):
    boot_amph = np.random.choice(amph, size=len(amph), replace=True)
    boot_ctrl = np.random.choice(control, size=len(control), replace=True)
    boot_diffs.append(boot_amph.mean() - boot_ctrl.mean())

boot_diffs = np.array(boot_diffs)

print("BOOTSTRAP ANALYSIS (10,000 resamples)")
print("=" * 45)
print(f"Amphetamine vs Control difference")
print(f"\nObserved difference: {amph.mean() - control.mean():.1f} squares")
print(f"Bootstrap mean:      {np.mean(boot_diffs):.1f} squares")
print(f"95% CI: [{np.percentile(boot_diffs, 2.5):.1f}, {np.percentile(boot_diffs, 97.5):.1f}]")
print(f"\nP(Amph > Control): {(boot_diffs > 0).mean():.1%}")

# Visualize
plt.figure(figsize=(8, 4))
plt.hist(boot_diffs, bins=50, edgecolor='black', alpha=0.7, color='#E41A1C')
plt.axvline(0, color='black', linewidth=2, linestyle='--', label='No difference')
plt.axvline(np.percentile(boot_diffs, 2.5), color='blue', linewidth=2, linestyle=':', label='95% CI')
plt.axvline(np.percentile(boot_diffs, 97.5), color='blue', linewidth=2, linestyle=':')
plt.xlabel('Amphetamine − Control (squares entered)')
plt.ylabel('Count')
plt.title('Bootstrapped Distribution: Amphetamine Effect')
plt.legend()
plt.tight_layout()
plt.show()

### Advantage 4: Reproducibility

If someone asks *"How did you get that result?"*, you can hand them this notebook. Every step is documented, every analysis is re-runnable.

In JASP, you'd have to write out: *"I clicked ANOVA, then dragged Squares entered into the dependent variable box, then I dragged Group into the grouping variable, then I checked the Tukey option under Post Hoc..."*

In Python, the code **is** the documentation.

---

## Common Errors and What They Mean

When you see an error, don't panic! Here's what common errors mean:

| Error | What it means | How to fix |
|-------|---------------|------------|
| `FileNotFoundError` | Python can't find the file | Check filename spelling, make sure file is in the same folder as the notebook |
| `KeyError: 'column_name'` | That column doesn't exist in your data | Check spelling with `df.columns` to see all column names |
| `ValueError: could not convert string to float` | There's text in a column that should be numbers | Check for header rows or non-numeric data with `df.head()` |
| `NameError: name 'x' is not defined` | You're using a variable before creating it | Make sure you ran the cell that creates that variable first |
| `IndentationError` | Python code isn't lined up correctly | Check that spaces/tabs are consistent |

**Pro tip:** Read error messages from the bottom up — the last line usually tells you what went wrong!

---

## Bonus: Repeated Measures ANOVA (Optional)

Repeated measures designs (same subjects measured multiple times) require special handling. Python can do this using the `pingouin` library, but it requires additional setup.

**This section is optional and won't be covered in today's lab.**

If you want to explore on your own:
1. Install pingouin: `pip install pingouin`
2. See the documentation: [pingouin.rm_anova](https://pingouin-stats.org/generated/pingouin.rm_anova.html)

Example code (don't run unless you have pingouin installed):

```python
import pingouin as pg

# Repeated measures ANOVA
pg.rm_anova(data=df, dv='Score', within='Time', subject='Subject')

# Mixed ANOVA (within + between factors)
pg.mixed_anova(data=df, dv='Score', within='Time', between='Group', subject='Subject')
```

---

## Checkpoint: JASP vs Python

You just replicated **every test** from the JASP lab in Python. Let's count the lines of code:

| Test | Python Code | JASP |
|------|-----------|------|
| Independent t-test | `stats.ttest_ind(a, b)` — **1 line** | 4-5 clicks, drag variables |
| Paired t-test | `stats.ttest_rel(a, b)` — **1 line** | Reshape data, 4-5 clicks |
| Welch's t-test | `stats.ttest_ind(a, b, equal_var=False)` — **1 line** | Buried in options menu |
| Levene's test | `stats.levene(a, b)` — **1 line** | Separate analysis |
| One-way ANOVA | `stats.f_oneway(g1, g2, ...)` — **1 line** | 3-4 clicks, check boxes |
| Tukey HSD | `tukey_hsd(g1, g2, ...)` — **1 line** | Check post-hoc options |
| 28 pairwise tests | Loop: 8 lines total | Click 28× through menus |

The tests are equally simple. **But Python can do things JASP cannot.** We demonstrated:
- Automation (loop through all comparisons)
- Custom publication-quality visualizations
- Bootstrap confidence intervals
- Complete reproducibility

---

## Summary: The Python Cheat Sheet

| What you want to do | Python code |
|---|---|
| Load Excel data | `df = pd.read_excel('file.xlsx')` |
| Load CSV data | `df = pd.read_csv('file.csv')` |
| View first rows | `df.head()` |
| Check data structure | `df.info()` |
| Check for missing values | `df.isna().sum()` |
| Group means | `df.groupby('Group')['DV'].mean()` |
| Descriptive stats | `df.groupby('Group')['DV'].agg(['count', 'mean', 'std'])` |
| Independent t-test | `stats.ttest_ind(group1, group2)` |
| Welch's t-test | `stats.ttest_ind(group1, group2, equal_var=False)` |
| Paired t-test | `stats.ttest_rel(pre, post)` |
| Levene's test | `stats.levene(group1, group2)` |
| One-way ANOVA | `stats.f_oneway(g1, g2, g3, ...)` |
| Tukey HSD | `tukey_hsd(g1, g2, g3, ...)` |
| Bar plot | `plt.bar(x, heights, yerr=errors)` |
| Histogram | `plt.hist(x, bins=30)` |
| Boxplot | `plt.boxplot([g1, g2], labels=['A', 'B'])` |

**That's it.** With these functions, you can do everything JASP does — and much more.